In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from keras.applications import VGG16

import cv2 as cv

import zipfile

from keras import models,layers
import os,shutil
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers,metrics

from collections import Counter


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [14]:
[len(os.listdir('../dataset/train/'+i)) for i in os.listdir('../dataset/train/')] 

[7383,
 1124,
 706,
 1124,
 4104,
 4575,
 615,
 7952,
 5568,
 4771,
 3377,
 7143,
 2642,
 591,
 4364]

In [15]:
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(100, 100, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
#  conv_base.trainable = False

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(15, activation='softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2359808   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                7695      
Total params: 17,082,191
Trainable params: 9,446,927
Non-trainable params: 7,635,264
_________________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [20]:
x_train, y_train = [], []
for i in os.listdir('../dataset/train'):
  for j in os.listdir('../dataset/train/'+i):
    x_train.append(cv.resize(cv.imread('../dataset/train/'+i+'/'+j),(100,100)))
    y_train.append(int(i))
    
x_train, y_train = np.array(x_train),np.array(y_train)
x_train.shape,y_train.shape

((56039, 100, 100, 3), (56039,))

In [21]:
x_test, y_test = [], []
for i in os.listdir('../dataset/Validation'):
  for j in os.listdir('../dataset/Validation/'+i):
    x_test.append(cv.resize(cv.imread('../dataset/Validation/'+i+'/'+j),(100,100)))
    y_test.append(int(i))
    
x_test, y_test = np.array(x_test),np.array(y_test)
x_test.shape,y_test.shape

((6219, 100, 100, 3), (6219,))

In [22]:
test = np.array([cv.resize(cv.imread('../dataset/test/'+i),(100,100)) for i in os.listdir('../dataset/test')])

test.shape

(21273, 100, 100, 3)

In [23]:
gc.collect()
# del x_train,y_train,x_test,y_test

3889

In [0]:
x_train = x_train.reshape(52925, 100, 100, 3)
x_test = x_test.reshape(9333, 100, 100, 3)
test = test.reshape(21273, 100, 100, 3)

In [0]:
from keras.utils import to_categorical,normalize

train_labels = to_categorical(y_train)
validation_labels = to_categorical(y_test)

In [25]:
gc.collect()

0

In [36]:
model.fit(x_train, train_labels, epochs=10, batch_size=512,validation_data=(x_test, validation_labels))

Train on 56039 samples, validate on 6219 samples
Epoch 1/10
56039/56039 [==============================] - 158s 3ms/step - loss: 1.2312 - acc: 0.5567 - val_loss: 1.9100 - val_acc: 0.4554
Epoch 2/10
56039/56039 [==============================] - 157s 3ms/step - loss: 1.2119 - acc: 0.5649 - val_loss: 1.8465 - val_acc: 0.4522
Epoch 3/10
56039/56039 [==============================] - 157s 3ms/step - loss: 1.2047 - acc: 0.5651 - val_loss: 1.8806 - val_acc: 0.4570
Epoch 4/10
 4608/56039 [=>............................] - ETA: 2:11 - loss: 1.1682 - acc: 0.5775

56039/56039 [==============================] - 157s 3ms/step - loss: 1.1941 - acc: 0.5678 - val_loss: 1.8645 - val_acc: 0.4546
Epoch 5/10
56039/56039 [==============================] - 157s 3ms/step - loss: 1.1812 - acc: 0.5716 - val_loss: 1.8684 - val_acc: 0.4554
Epoch 6/10
56039/56039 [==============================] - 157s 3ms/step - loss: 1.1757 - acc: 0.5734 - val_loss: 1.9091 - val_acc: 0.4605
Epoch 7/10
26624/56039 [=============>................] - ETA: 1:15 - loss: 1.1581 - acc: 0.5776

56039/56039 [==============================] - 157s 3ms/step - loss: 1.1609 - acc: 0.5779 - val_loss: 1.9159 - val_acc: 0.4570
Epoch 8/10
56039/56039 [==============================] - 157s 3ms/step - loss: 1.1466 - acc: 0.5825 - val_loss: 1.9643 - val_acc: 0.4584
Epoch 9/10
56039/56039 [==============================] - 157s 3ms/step - loss: 1.1363 - acc: 0.5854 - val_loss: 1.9308 - val_acc: 0.4607
Epoch 10/10
30208/56039 [===============>..............] - ETA: 1:05 - loss: 1.1372 - acc: 0.5855

56039/56039 [==============================] - 157s 3ms/step - loss: 1.1281 - acc: 0.5881 - val_loss: 1.9492 - val_acc: 0.4594


In [0]:
model.save('../dataset/CNN_3.h5')

In [0]:
model.fit(x_train, train_labels, epochs=50, batch_size=512,validation_data=(x_test, validation_labels))

Train on 56039 samples, validate on 6219 samples
Epoch 1/50
56039/56039 [==============================] - 158s 3ms/step - loss: 0.9324 - acc: 0.6515 - val_loss: 2.2392 - val_acc: 0.4657
Epoch 2/50
 1536/56039 [..............................] - ETA: 2:19 - loss: 0.9846 - acc: 0.6270

In [39]:
gc.collect()

319

In [0]:
sub = pd.DataFrame()
sub['image_name'] = [i for i in os.listdir('../dataset/test')]

In [41]:
sub.head()

,image_name
0,n03404251_4832_1.jpg
1,n03228254_5787_0.jpg
2,n04602956_2854_0.jpg
3,n03467254_1056_0.jpg
4,n04355511_1909_0.jpg


In [0]:
sub['category'] = model.predict_classes(test)

In [0]:
sub.to_csv('../dataset/submission_pre_vgg_3.csv',index=False)

In [0]:
from google.colab import files
files.download('../dataset/submission_pre_vgg_3.csv')

In [26]:
# *********************************************************************************************************************************************************
#															Image Augmentation
# *********************************************************************************************************************************************************
from keras.preprocessing import image

# set limit_1 and limit_2 such that augmentation will take place only when no of image will be less than limit_x

def augment_big_data_image(src = '', dest = '', limit_1 = 0, no_of_aug_1 = 0, limit_2 = 0, no_of_aug_2 = 0):
  print('Before augmentation : ')  
  for i in os.listdir(src):
    print(i,'  -  ',len([j for j in os.listdir(src+'/'+i)]))
    	
	# data generator
  datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, 
                               shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
	
	for jk in os.listdir(src):
    print('Augmenting ',jk)
    if len([i for i in os.listdir(src+'/'+jk)]) < limit_1:
      for i in os.listdir(src+'/'+jk):
        img = cv.resize(cv.imread(src+'/'+jk+'/'+i))
  
  
  
  for jk in os.listdir(src):
    print('Augmenting ',jk)
    if len([i for i in os.listdir(src+'/'+jk)]) <limit_1:
      for i in os.listdir(src+'/'+jk):
        img = cv.resize(cv.imread(src+'/'+jk+'/'+i,cv.IMREAD_GRAYSCALE),(150,150))
				cntr = 0
				for batch in datagen.flow(img.reshape(1,150,150,1), batch_size=1):
          cv.imwrite(dest+'/'+jk+'/'+str(cntr)+i,batch[0].reshape(150,150))
					cntr+=1
					if cntr>no_of_aug_1:
						break
		elif  len([i for i in os.listdir(src+'/'+jk)]) < limit_2 :
			for i in os.listdir(src+'/'+jk):
				img = cv.resize(cv.imread(src+'/'+jk+'/'+i,cv.IMREAD_GRAYSCALE),(150,150))
			  cntr = 0
				for batch in datagen.flow(img.reshape(1,150,150,1), batch_size=1):
					cv.imwrite(dest+'/'+jk+'/'+str(cntr)+i,batch[0].reshape(150,150))
					cntr+=1
					if cntr>no_of_aug_2:
						break
						
	# move data to src location
	for i in os.listdir(dest):
		if len([j for j in os.listdir(dest+'/'+i)]) >0:
			for k in os.listdir(dest+'/'+i):
				shutil.move(dest+'/'+i+'/'+k,src+'/'+i+'/'+k)
				
	print('After augmentation : ')
	for i in os.listdir(src):
		print(i,' -  ',len([j for j in os.listdir(src+'/'+i)]))

TabError: ignored

In [14]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Gen RAM Free: 10.4 GB  I Proc size: 2.4 GB
GPU RAM Free: 564MB | Used: 10875MB | Util  95% | Total 11439MB
